# start

In [1]:
PRFX='0318_6'
# http://localhost:8081/notebooks/git/recsys20/working/prep0318_1.ipynb
PRFX_PRP = 'prep0318_1' 
langs = ['22C448FF81263D4BAF2A176145EE9EAD',]
trntmstmp=1584412344
import datetime
[datetime.datetime.fromtimestamp(o).strftime('%Y-%m-%d %H:%M:%S') for o in (trntmstmp, valtmstmp)]

['2020-03-17 02:32:24', '2020-03-17 00:30:47']

# setup

In [2]:
from pathlib import Path
import dask
print('dask.__version__', dask.__version__)
import xgboost
import dask_xgboost
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from dask.distributed import Client
import pickle
import lightgbm as lgb

import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_prp=f'{HOME}/output/{PRFX_PRP}'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
    prec, recall, thresh = precision_recall_curve(gt, pred)
    prauc = auc(recall, prec)
    return prauc

def calculate_ctr(gt):
    positive = len([x for x in gt if x == 1])
    ctr = positive/float(len(gt))
    return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwing_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_follower_cnt',
'u2_following_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]

cols_val = cols[:-4]

tgts     = ['Retweet','Reply','Like','RTwCmnt',]
cols_tgt = ['retwt',  'reply','like','retwt_cmmnt',]


dask.__version__ 2.12.0


/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/array/random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


# prepare data

In [3]:
ls -hlS $p_prp | grep dftrn | head

-rw-rw-r-- 1 ubuntu ubuntu  15G Mar 18 20:45 dftrn_22C448FF81263D4BAF2A176145EE9EAD_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  35M Mar 18 13:41 dftrn_3A85BCEC571C3F5AB1069E4924189177_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  34M Mar 18 13:41 dftrn_69C4A33B9AD29AF883D60BA61CC08702_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  30M Mar 18 13:41 dftrn_CB11E9CF42BD0A1BAD5E27BF3422D99D_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  26M Mar 18 13:41 dftrn_E7BB61D2A87C1E72DF1C7BC292B86A1C_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  23M Mar 18 13:41 dftrn_FF7EABB5A382356D54D9C41BA0125E09_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  20M Mar 18 13:42 dftrn_54208B51D44E7D91DC2F3DD02ADEDEC2_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  19M Mar 18 13:42 dftrn_B6D90127A09AB1229731898AEF9D4D7C_1584412344__prep0318_1.tsv
-rw-rw-r-- 1 ubuntu ubuntu  17M Mar 18 13:41 dftrn_F4FD40A716F1572C9A28E9CAA58BE3A5_1584412344__prep0318_1.tsv
-

In [4]:
df = dd.concat([dd.read_csv(f'{p_prp}/dftrn_{lang}_{trntmstmp}__{PRFX_PRP}.tsv',sep='\x01') for lang in langs])

In [5]:
%%time
# https://stackoverflow.com/questions/40376425/dask-dataframe-equivalent-of-pandas-dataframe-sort-values
lendf = df.shape[0].compute()
valsz = int(lendf*0.15)
trnsz = lendf-valsz
lendf, trnsz, valsz

CPU times: user 6min 3s, sys: 40.7 s, total: 6min 44s
Wall time: 2min 50s


(23652265, 20104426, 3547839)

In [ ]:
%%time
dftr=df.nsmallest(trnsz,'tm')
dfvl=df.nlargest(valsz,'tm')

# assert dfvl.tm.min().compute() >= dftr.tm.max().compute()

In [ ]:
dftr.columns

In [ ]:
cols_feat=[
       'tm', 'u1_fllwer_cnt', 'u1_fllwing_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2_follower_cnt', 'u2_following_cnt',
       'u2_vrfed', 'u2_create_tm', 'u1_fllw_u2', 'len_toks', 'has_media_Photo',
       'has_media_Video', 'has_media_GIF', 'num_hshtgs', 'num_links',
       'num_domns', 'twttyp_TopLevel', 'twttyp_Retweet', 'twttyp_Quote',
       'tmdlta_u2u1', 'tmdlta_twtu1', 'tmdlta_twtu2',
]

In [ ]:
%%time
X, ys = (df[cols].to_dask_array(lengths=True) for cols in (cols_feat,cols_tgt))

In [ ]:
%%time
Xtrn, Xval, ystrn, ysval = train_test_split(X, ys, test_size=0.15, random_state=SEED)

# train

In [ ]:
client = Client()

In [ ]:
tgt2params = {
        'Retweet': {
            'objective': 'binary:logistic',
#             'max_depth':5,
#             'min_child_weight':2,
#             'gamma':1,
        }, 
        'Reply': {
            'objective': 'binary:logistic',
#             'max_depth':5,
#             'min_child_weight':2,
            'gamma':1,
        }, 
        'Like': {
            'objective': 'binary:logistic',
#             'max_depth':5,
#             'min_child_weight':2,
#             'gamma':1,
        }, 
        'RTwCmnt': {
            'objective': 'binary:logistic',
#             'max_depth':5,
#             'min_child_weight':2,
#             'gamma':1,
        }, 
}

In [ ]:
tgt2bst={}
tgt2col2fscr={}
for i,tgt in enumerate(tgts):
    params = tgt2params[tgt]
    bst = dask_xgboost.train(client, params,
                             Xtrn, ystrn[:,i],
                             num_boost_round=10)
    tgt2bst[tgt]=bst
    print(dtnow(), tgt)
    
    fi2col=dict(zip(bst.feature_names, cols_feat))
    col2fi=dict(zip(cols_feat, bst.feature_names))
    fi2fscr=bst.get_fscore()
    col2fscr={col:fi2fscr.get(fi,0) for col,fi in col2fi.items()}
    tgt2col2fscr[tgt]=col2fscr

pickle.dump(tgt2bst, open(f"{p_out}/tgt2bst.p", "wb"))

In [ ]:
tgt2bst=pickle.load(open(f"{p_out}/tgt2bst.p", "rb"))

In [ ]:
def show_feat_importance(bst):
    ax = xgboost.plot_importance(bst, height=0.8, max_num_features=9)
    ax.grid(False, axis="y")
    ax.set_title('Estimated feature importance')
    plt.show()

for i,tgt in enumerate(tgts):
    print(tgt)
    show_feat_importance(tgt2bst[tgt])
    col2fscr=tgt2col2fscr[tgt]
    display(pd.DataFrame(col2fscr.items()).sort_values(1, ascending=False))

# validate

In [ ]:
tgt2ytrn={}
tgt2yval={}
tgt2prdtrn={}
tgt2prdval={}
for i,tgt in enumerate(tgts):
    print(dtnow(), tgt)
    prdtrn = dask_xgboost.predict(client, tgt2bst[tgt], Xtrn).persist()
    prdval = dask_xgboost.predict(client, tgt2bst[tgt], Xval).persist()
    ytrn, prdtrn = dask.compute(ystrn[:,i], prdtrn)
    yval, prdval = dask.compute(ysval[:,i], prdval)
    tgt2ytrn[tgt],tgt2yval[tgt]=ytrn,yval
    tgt2prdtrn[tgt],tgt2prdval[tgt]=prdtrn,prdval

In [ ]:
def show_roc_auc(prd,y):
    fpr, tpr, _ = roc_curve(y,prd)
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.plot(fpr, tpr, lw=3,
            label='ROC Curve (area = {:.2f})'.format(auc(fpr, tpr)))
    ax.plot([0, 1], [0, 1], 'k--', lw=2)
    ax.set(
        xlim=(0, 1),
        ylim=(0, 1),
        title="ROC Curve",
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
    )
    ax.legend();
    plt.show()

In [ ]:
tgt2auc_trn={}
tgt2rce_trn={}
tgt2auc_val={}
tgt2rce_val={}
for i,tgt in enumerate(tgts):
    print(tgt)
    ytrn,yval = tgt2ytrn[tgt],tgt2yval[tgt]
    prdtrn,prdval = tgt2prdtrn[tgt],tgt2prdval[tgt]
    scr_rocauc_trn = roc_auc_score(ytrn, prdtrn)
    scr_rocauc_val = roc_auc_score(yval, prdval)
    scr_auc_trn=compute_prauc(prdtrn, ytrn)
    scr_rce_trn=compute_rce(prdtrn, ytrn)
    scr_auc_val=compute_prauc(prdval, yval)
    scr_rce_val=compute_rce(prdval, yval)
    tgt2auc_trn[tgt]=scr_auc_trn
    tgt2rce_trn[tgt]=scr_rce_trn
    tgt2auc_val[tgt]=scr_auc_val
    tgt2rce_val[tgt]=scr_rce_val
    print('train rocauc:', f'{scr_rocauc_trn:.4f}', 'valid auc:', f'{scr_rocauc_val:.4f}', )
    print('train prauc:', f'{scr_auc_trn:.4f}', 'valid auc:', f'{scr_auc_val:.4f}', )
    print('train rce:', f'{scr_rce_trn:.4f}', 'valid rce:', f'{scr_rce_val:.4f}', )

In [ ]:
lsttrn=[]
lstval=[]
for tgt in ['Retweet','Reply','Like','RTwCmnt',]:
    lsttrn+=[(f'PRAUC {tgt}',tgt2auc_trn[tgt]),
          (f'RCE {tgt}',tgt2rce_trn[tgt])]
    lstval+=[(f'PRAUC {tgt}',tgt2auc_val[tgt]),
          (f'RCE {tgt}',tgt2rce_val[tgt])]

dfscrtrn=pd.DataFrame(lsttrn)
dfscrtrn.columns=['metric','scr']
dfscrval=pd.DataFrame(lstval)
dfscrval.columns=['metric','scr']

In [ ]:
dfscr = pd.merge(dfscrtrn, dfscrval, on='metric', suffixes=('trn','val'))
dfscr.T

# infer

In [ ]:
PRFX_PRP_TST = 'prep0318_2' 
valtmstmp=

In [ ]:
ls {p_prp} | grep {valtmstmp}

In [ ]:
dftst=dd.read_csv(
    f'{p_prp}/dfval_{valtmstmp}__{PRFX_PRP_TST}.tsv',
    sep='\x01',
    encoding='utf8')

In [ ]:
Xtst = dftst[cols_feat].to_dask_array(lengths=True)

In [ ]:
%%time
tgt2prdtst={}
for i,tgt in enumerate(tgts):
    prdtst = dask_xgboost.predict(client, tgt2bst[tgt], Xtst)
    prdtst = prdtst
    tgt2prdtst[tgt]=prdtst 

In [ ]:
dfsub_ids = dftst[['twtid','u2id',]]

tgt2dfsub = {}
for tgt,prdtst in tgt2prdtst.items():
    dfsub = dfsub_ids.copy()
    dfsub['scr'] = prdtst
    tgt2dfsub[tgt]=dfsub

In [ ]:
%%time
for tgt, dfsub in tgt2dfsub.items():
    print(dtnow(), tgt)
    dfsub.to_csv(f'{p_out}/{tgt}__{valtmstmp}__{PRFX}.csv',index=False,header=False,single_file=True)

In [ ]:
# %%time
# for tgt in tgts:
#     print(dtnow(), tgt)
#     dfsub = pd.concat(pd.read_csv(o, header=None) for o in sorted(glob.glob(f'{p_out}/{tgt}__{PRFX}_*.csv')))
#     dfsub.to_csv(f'{p_out}/{tgt}__{PRFX}.csv', 
#                           index=False, 
#                           header=False)

In [ ]:
!head -n 2 {p_prp}/val_{valtmstmp}.tsv

In [ ]:
for tgt in tgts:
    !head -n 2 {p_out}/{tgt}__{valtmstmp}__{PRFX}.csv